## 1.4 Les limites de PEP249

<p style="font-size: 120%">
Un package donné indique le format retenu pour les requêtes paramétrées à l'aide de l'attribut <code>paramstyle</code>&#160;:<br>
<a href="https://peps.python.org/pep-0249/#paramstyle">https://peps.python.org/pep-0249/#paramstyle</a>
</p>

In [1]:
import sqlite3
print(sqlite3.paramstyle)

qmark


In [2]:
import mysql.connector
print(mysql.connector.paramstyle)

pyformat


<p style="font-size: 120%">
La syntaxe des requêtes paramétrées n'est donc pas la même sous SQLite ou MySQL :
</p>

In [3]:
import sqlite3
conn = sqlite3.connect('my_database.sqlite')
c = conn.cursor()

pseudo = 'rde'
password = 'hello'

request = "SELECT * FROM users WHERE pseudo=? AND pwd=?"
c.execute(request,(pseudo,password))

print(c.fetchone())
conn.close()


('rde', 'Raymond.Deubaze@ec-lyon.fr', 'hello')


In [4]:
conn = mysql.connector.connect(
    host="localhost",
    user="my_database",
    password="pwd",
    database="my_database"
)
c = conn.cursor()

pseudo = 'rde'
password = 'hello'

request = "SELECT * FROM users WHERE pseudo=%s AND pwd=%s"
c.execute(request,(pseudo,password))

print(c.fetchone())
conn.close()

('rde', 'Raymond.Deubaze@gmail.com', 'hello')


<p style="font-size: 120%">
<b>N.B.</b> Dans le cas du driver MySQL utilisé, on peut utiliser la notation <code>%s</code> quels que soient les types python et SQL (cf. <a href="https://gayerie.dev/docs/python/python3/mysql.html#insertion-de-donnees">la doc</a>).
</p>

<pre style="background-color:#f0f0f0; border: 1px solid #ccc; margin:0; padding:0.5em; font-size:1em">
conn = mysql.connector.connect(
    host="localhost",
    user="my_database",
    password="pwd",
    database="my_database"
)
c = conn.cursor()


request = "INSERT INTO test_values (id,name,value) VALUES (%s,%s,%s)"
retval = c.execute(request,(1,'pi',3.141592))

print(retval)
conn.close()
</pre>

### Code générique

<p style="font-size: 120%">
La construction de code générique pour ces deux drivers est possible, mais nécessite une petite gymnastique...
</p>

In [5]:
def build_select(driver,sql,numargs):
    s = { 'qmark' : '?', 'pyformat': '%s'}
    return sql.format(*([s[driver.paramstyle]]*numargs))

args = ('rde','hello')
sql = 'SELECT * FROM users WHERE pseudo={} AND pwd={}'

In [6]:
import sqlite3
driver = sqlite3

request = build_select(driver, sql, len(args))
print(request)

SELECT * FROM users WHERE pseudo=? AND pwd=?


In [7]:
import mysql.connector
driver = mysql.connector

request = build_select(driver, sql, len(args))
print(request)

SELECT * FROM users WHERE pseudo=%s AND pwd=%s


<p style="font-size: 120%">
Toutefois, <code>paramstyle</code> permet 5 approches différentes dont seulement 2 ont été illustrées ici, sans préjuger de l'interprétation que peuvent en faire les divers drivers. Le construction de code agnostique et totalement générique n'est donc pas du tout triviale...
</p>